# SIIM MMDetection train demo notebook

This notebook uses [MMDetection](https://github.com/open-mmlab/mmdetection) framework to identify opacity bounding boxes in the SIIM competition.

[This dataset](https://www.kaggle.com/xhlulu/siim-covid19-resized-to-512px-png) is used for the train process, so please upvote it.

## Install dependencies

In [ ]:
!pip install ../input/mmdet2100/mmdetection-2.10.0/addict-2.4.0-py3-none-any.whl
!pip install ../input/mmdet2100/mmdetection-2.10.0/yapf-0.31.0-py2.py3-none-any.whl
!pip install ../input/pycocotools202/pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl
!pip install ../input/mmcvfull134/mmcv_full-1.3.4-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/mmdetection2120/mmdetection-2.12.0 -f ./ --no-index
!pip install ../input/ensemble-boxes-104/ensemble_boxes-1.0.4/ -f ./ --no-index

In [ ]:
import os
import shutil
import yaml
import time
import json
import cv2
import random
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold, StratifiedKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch
from IPython.display import Image, clear_output
from collections import Counter
from ensemble_boxes import *
import copy
import os.path as osp
import mmcv
import mmdet
import numpy as np
import albumentations as A
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
print(mmdet.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
VER = 'v4_4'
DEBUG = False
PARAMS = {
    'version': VER,
    'folds': 5,
    'val_fold': 4,
    'img_size': 640,
    'batch_size': 8,
    'epochs': 16,
    'seed': 2021,
    'iou_th': .6,
    'th': .4,
    ### r50
    'config': 'vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco.py',
    'checkpoint': 'vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-6879c318.pth',
    ### r101
    #'config': 'vfnet_r101_fpn_mdconv_c3-c5_mstrain_2x_coco.py',
    #'checkpoint': 'vfnet_r101_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-7729adb5.pth',
    'comments': ''
}
DATA_PATH = '/kaggle/input/siim-covid19-detection'
IMGS_PATH = f'/kaggle/input/siim-covid19-resized-384512-and-640px/SIIM-COVID19-Resized/img_sz_{PARAMS["img_size"]}'
#'/kaggle/input/siim-covid19-resized-to-512px-png'
CHKP_PATH = '/kaggle/input/mmdet-vfnet-pretrained'
MDLS_PATH = f'/kaggle/working/models_mmdet_{VER}'
if not os.path.exists(MDLS_PATH):
    os.mkdir(MDLS_PATH)
with open(f'{MDLS_PATH}/params.json', 'w') as file:
    json.dump(PARAMS, file)
    
def seed_all(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_all(PARAMS['seed'])
start_time = time.time()

## Data preprocess

In [ ]:
train_df = pd.read_csv(f'{IMGS_PATH}/meta_sz_{PARAMS["img_size"]}.csv')
train_df = train_df[train_df.split == 'train']
del train_df['split']
if DEBUG:
    train_df = train_df.loc[:100]
df_train_img = pd.read_csv(f'{DATA_PATH}/train_image_level.csv')
df_train_sty = pd.read_csv(f'{DATA_PATH}/train_study_level.csv')

train_df['id'] = train_df['image_id'].apply(lambda x: ''.join([x.split('/')[-1], '_image']))
df_train_sty['StudyInstanceUID'] = df_train_sty['id'].apply(lambda x: x.replace('_study', ''))
del df_train_sty['id']
df_train_img = df_train_img.merge(df_train_sty, on='StudyInstanceUID')
train_df = df_train_img.merge(train_df, on='id')
train_df['img'] = train_df['image_id'] + '.jpg'
print(train_df.shape)
display(train_df.head())

In [ ]:
def bar_plot(train_df, variable):
    var = train_df[variable]
    varValue = var.value_counts()
    plt.figure(figsize = (12, 3))
    plt.bar(varValue.index, varValue)
    plt.xticks(varValue.index, varValue.index.values)
    plt.ylabel("Frequency")
    plt.title(variable)
    plt.show()
    print("{}: \n{}".format(variable, varValue))

train_df['target'] = 'Negative for Pneumonia'
train_df.loc[train_df['Typical Appearance']==1, 'target'] = 'Typical Appearance'
train_df.loc[train_df['Indeterminate Appearance']==1, 'target'] = 'Indeterminate Appearance'
train_df.loc[train_df['Atypical Appearance']==1, 'target'] = 'Atypical Appearance'
bar_plot(train_df, 'target') 

In [ ]:
train_df = train_df[~train_df.boxes.isnull()] 
train_df.reset_index(inplace=True)
classes = [
    'Typical Appearance', 
    'Indeterminate Appearance', 
    'Atypical Appearance'
]
print('classes:\n', classes,
      '\nclasses labels:\n', np.unique(train_df[classes].values, axis=0))

In [ ]:
label2color = {
    '[1, 0, 0]': [255, 0, 0], # Typical Appearance
    '[0, 1, 0]': [0, 255, 0], # Indeterminate Appearance
    '[0, 0, 1]': [0, 0, 255], # Atypical Appearance
}
label2classes = {
    '[1, 0, 0]': classes[0],
    '[0, 1, 0]': classes[1],
    '[0, 0, 1]': classes[2]
}

def plot_img(img, size=(18, 18), title='', cmap='gray'):
    plt.figure(figsize=size)
    plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()

def plot_imgs(imgs, cols=2, size=10, is_rgb=True, title='', cmap='gray', img_size=None):
    rows = len(imgs) // cols + 1
    fig = plt.figure(figsize=(cols * size, rows * size))
    for i, img in enumerate(imgs):
        if img_size is not None:
            img = cv2.resize(img, img_size)
        fig.add_subplot(rows, cols, i + 1)
        plt.axis('off')
        plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.axis('off')
    
def draw_bbox(img, box, label, color, thickness=3):   
    alpha = .1
    alpha_box = .4
    overlay_bbox = img.copy()
    overlay_text = img.copy()
    output = img.copy()
    text_width, text_height = cv2.getTextSize(label.upper(), cv2.FONT_HERSHEY_SIMPLEX, .6, 1)[0]
    cv2.rectangle(overlay_bbox, 
                  (box[0], box[1]), 
                  (box[2], box[3]), 
                  color, -1)
    cv2.addWeighted(overlay_bbox, alpha, output, 1 - alpha, 0, output)
    cv2.rectangle(overlay_text, 
                  (box[0], box[1] - 7 - text_height), 
                  (box[0] + text_width + 2, box[1]),
                  (0, 0, 0), -1)
    cv2.addWeighted(overlay_text, alpha_box, output, 1 - alpha_box, 0, output)
    cv2.rectangle(output, 
                  (box[0], box[1]), 
                  (box[2], box[3]),
                  color, thickness)
    cv2.putText(output, 
                label.upper(), 
                (box[0], box[1]-5),
                cv2.FONT_HERSHEY_SIMPLEX, 
                .6, (255, 255, 255), 1, 
                cv2.LINE_AA)
    return output

In [ ]:
imgs = []
sample = train_df.sample(n=4)['img'].values
for img_name in sample:
    ratio_x = PARAMS['img_size'] / train_df.loc[train_df['img'] == img_name, 'dim1'].values[0]
    ratio_y = PARAMS['img_size'] / train_df.loc[train_df['img'] == img_name, 'dim0'].values[0]
    boxes = train_df.loc[train_df['img'] == img_name, 'boxes'].values[0]
    boxes = json.loads(boxes.replace('\'', '\"'))
    boxes = [[int(box['x'] * ratio_x), 
              int(box['y'] * ratio_y), 
              int((box['x'] + box['width']) * ratio_x), 
              int((box['y'] + box['height']) * ratio_y)]
             for box in boxes]
    img_labels = train_df.loc[train_df['img'] == img_name, classes].values[0]
    img_labels = [str(img_labels.tolist())] * len(boxes)
    img = cv2.imread(f'{IMGS_PATH}/train/{img_name}')
    for label_id, box in zip(img_labels, boxes):
        color = label2color[label_id]
        img = draw_bbox(
            img, 
            list(np.int_(box)), 
            label2classes[label_id], 
            label2color[label_id]
        )
    imgs.append(img)
plot_imgs(imgs, size=8, cols=4, cmap=None)

In [ ]:
skf  = StratifiedKFold(n_splits=PARAMS['folds'])
train_df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, y=train_df.target)):
    train_df.loc[val_idx, 'fold'] = fold

In [ ]:
split = PARAMS['val_fold']
with open(f'{MDLS_PATH}/train.txt', 'w') as file:
    tr_ids = list(train_df[train_df['fold'] != split].img.unique())
    print('train:', len(tr_ids))
    file.write('\n'.join(tr_ids))
with open(f'{MDLS_PATH}/val.txt', 'w') as file:
    val_ids = list(train_df[train_df['fold'] == split].img.unique())
    print('val:', len(val_ids))
    file.write('\n'.join(val_ids))

## Training

In [ ]:
@DATASETS.register_module()
class SIIMDataset(CustomDataset):
    CLASSES = ('opacity', )
    ANN_DF = train_df.copy()
    def load_annotations(self, ann_file):
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)
        data_infos = []
        for img_id in image_list:
            img_anns = self.ANN_DF[self.ANN_DF.img == img_id]
            filename = f'{self.img_prefix}/{img_anns["img"].values[0]}'
            data_info = dict(
                filename=filename, 
                width=PARAMS['img_size'], 
                height=PARAMS['img_size']
            )
            ratio_x = PARAMS['img_size'] / img_anns['dim1'].values[0]
            ratio_y = PARAMS['img_size'] / img_anns['dim0'].values[0]
            boxes = img_anns['boxes'].values[0]
            boxes = json.loads(boxes.replace('\'', '\"'))
            gt_bboxes = [
                [int(box['x'] * ratio_x), 
                 int(box['y'] * ratio_y), 
                 int((box['x'] + box['width']) * ratio_x), 
                 int((box['y'] + box['height']) * ratio_y)]
                for box in boxes]
            img_labels = img_anns[classes].values[0]
            gt_labels = [0] * len(boxes)
            data_anno = dict(
                bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                labels=np.array(gt_labels),
            )
            data_info.update(ann=data_anno)
            data_infos.append(data_info)
        return data_infos

In [ ]:
train_transforms = A.Compose([
    A.OneOf([
        A.RandomBrightness(limit=.2, p=1), 
        A.RandomContrast(limit=.2, p=1), 
        A.RandomGamma(p=1)
    ], p=.5),
    A.OneOf([
        A.Blur(blur_limit=3, p=1),
        A.MedianBlur(blur_limit=3, p=1)
    ], p=.25),
    A.OneOf([
        A.GaussNoise(0.002, p=.5),
        A.IAAAffine(p=.5),
    ], p=.25),
    A.VerticalFlip(p=.5),
    A.HorizontalFlip(p=.5),
    A.Transpose(p=.25),
    A.RandomRotate90(p=.25),
    A.Cutout(num_holes=10, max_h_size=20, max_w_size=20, p=.25),
    A.ShiftScaleRotate(p=.5)
])

In [ ]:
cfg = Config.fromfile(f'../input/mmdetection2120/mmdetection-2.12.0/configs/vfnet/{PARAMS["config"]}')
cfg.load_from = f'{CHKP_PATH}/{PARAMS["checkpoint"]}'
cfg.model.bbox_head.num_classes = 1
cfg.dump(f'{MDLS_PATH}/init_config.py')
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=[(1333, 640), (1333, 672), (1333, 704), (1333, 736),
                   (1333, 768), (1333, 800)],
        multiscale_mode='value',
        keep_ratio=True),
    ########################################
    # Note that this key is part of bbox_params. 
    # Their difference is format='pascal_voc' means [x1, y1, x2, y2] style box encoding, 
    # while format='coco' means [x, y, w, h].
    dict(
        type='Albu',
        transforms=train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap={
            'img': 'image',
            'gt_bboxes': 'bboxes'},
        update_pad_shape=False,
        skip_img_without_anno=True),
    #########################################
    dict(
        type='Normalize',
        mean=[103.53, 116.28, 123.675],
        std=[1.0, 1.0, 1.0],
        to_rgb=False),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
cfg.dataset_type = 'SIIMDataset'
cfg.data_root = f'{IMGS_PATH}/train'
cfg.data.test.type = 'SIIMDataset'
cfg.data.test.data_root = IMGS_PATH
cfg.data.test.ann_file = f'{MDLS_PATH}/train.txt'
cfg.data.test.img_prefix = ''
cfg.data.train.type = 'SIIMDataset'
cfg.data.train.data_root = f'{IMGS_PATH}/train'
cfg.data.train.ann_file = f'{MDLS_PATH}/train.txt'
cfg.data.train.img_prefix = ''
cfg.data.val.type = 'SIIMDataset'
cfg.data.val.data_root = f'{IMGS_PATH}/train'
cfg.data.val.ann_file = f'{MDLS_PATH}/val.txt'
cfg.data.val.img_prefix = ''
cfg.work_dir = MDLS_PATH
cfg.optimizer.lr = .02 / (8 * 16 / PARAMS['batch_size'])
cfg.log_config.interval = 128
cfg.runner.max_epochs = PARAMS['epochs']
cfg.checkpoint_config.interval = 1
cfg.evaluation = dict(
    interval=1, 
    start=2,
    metric='mAP', 
    save_best='mAP')
cfg.seed = PARAMS['seed']
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.data.samples_per_gpu = PARAMS['batch_size']
cfg.data.workers_per_gpu = 2
cfg.workflow = [('train', 1)]
cfg.dump(f'{MDLS_PATH}/train_config.py')
print(f'Config:\n{cfg.pretty_text}')

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
datasets = [build_dataset(cfg.data.train)]
if len(cfg.workflow) == 2:
    datasets.append(build_dataset(cfg.data.val))
model = build_detector(
    cfg.model, 
    train_cfg=cfg.get('train_cfg'), 
    test_cfg=cfg.get('test_cfg')
)
model.CLASSES = datasets[0].CLASSES
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

## Inference demo

In [ ]:
checkpoint = f'{MDLS_PATH}/epoch_{PARAMS["epochs"]}.pth'
cfg = f'{MDLS_PATH}/init_config.py'
model_test = init_detector(cfg, checkpoint, device='cuda:0')

In [ ]:
imgs = []
split = PARAMS['val_fold']
sample = train_df[train_df['fold'] != split].sample(n=4)['img'].values
for img_name in sample:
    ratio_x = PARAMS['img_size'] / train_df.loc[train_df['img'] == img_name, 'dim1'].values[0]
    ratio_y = PARAMS['img_size'] / train_df.loc[train_df['img'] == img_name, 'dim0'].values[0]
    boxes = train_df.loc[train_df['img'] == img_name, 'boxes'].values[0]
    boxes = json.loads(boxes.replace('\'', '\"'))
    boxes = [[int(box['x'] * ratio_x), 
              int(box['y'] * ratio_y), 
              int((box['x'] + box['width']) * ratio_x), 
              int((box['y'] + box['height']) * ratio_y)]
             for box in boxes]
    img_labels = train_df.loc[train_df['img'] == img_name, classes].values[0]
    img_labels = [str(img_labels.tolist())] * len(boxes)
    img = cv2.imread(f'{IMGS_PATH}/train/{img_name}')
    for label_id, box in zip(img_labels, boxes):
        color = label2color[label_id]
        img = draw_bbox(
            img, 
            list(np.int_(box)), 
            label2classes[label_id], 
            label2color[label_id]
        )
    result = inference_detector(model_test, img)
    boxes_list = [list(x[:, :4] / PARAMS['img_size']) for x in result if x.shape[0] != 0]
    boxes_list =  [item for sublist in boxes_list for item in sublist]
    scores_list = [x[:, 4].tolist() for x in result if x.shape[0] != 0]
    scores_list =  [item for sublist in scores_list for item in sublist]
    labels_list = [[i] * x.shape[0] for i, x in enumerate(result) if x.shape[0] != 0]
    labels_list =  [item for sublist in labels_list for item in sublist]
    boxes, scores, box_labels = nms(
        boxes=[boxes_list], 
        scores=[scores_list], 
        labels=[labels_list], 
        weights=None,
        iou_thr=PARAMS['iou_th']
    )
    boxes *= PARAMS['img_size']
    for label_id, box, score in zip(box_labels, boxes, scores):
        if score >= PARAMS['th']:
            color = [255, 255, 255]
            img = draw_bbox(
                img, 
                list(np.int_(box)), 
                'predict', 
                color
            )
    imgs.append(img)
plot_imgs(imgs, size=8, cols=4, cmap=None)

In [ ]:
elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')